This notebook includes the process of fine-tuning the Meta-Llama-3-8B-Instruct model using LoRA (Low-Rank Adaptation). It covers setting up the environment with necessary libraries, loading the model with 4-bit quantization for memory efficiency. The notebook configures LoRA-specific parameters, sets up training arguments with Hugging Face's transformers, and uses the SFTTrainer.

In [ ]:
!pip install peft

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
dir = "meta-llama/Meta-Llama-3-8B-Instruct"
access_token = "YOUR_PINECONE_API_KEY" # Replace with your pinecone api key

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
import torch

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.9 MB/s eta 0:00:00


In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4"
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import bitsandbytes
from transformers import LlamaForCausalLM, Trainer, TrainingArguments, AutoTokenizer


model = LlamaForCausalLM.from_pretrained(dir, token = access_token,quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(dir, token=access_token)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
dataset = pd.read_excel('data_5983_final.xlsx')

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, val_test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# 50% validation, 50% test
val_dataset, test_dataset = train_test_split(val_test_dataset, test_size=0.5, random_state=42)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Training dataset size: 2246
Validation dataset size: 281
Test dataset size: 281


In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias= "none"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
val_dataset = val_dataset.sample(frac=1, random_state=42).iloc[:100]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned-llama-latest",
    evaluation_strategy="steps",
    eval_steps=1200,
    optim="paged_adamw_32bit",
    save_strategy="steps",
    save_steps=1200,
    max_steps=6000,
    learning_rate=6e-5,
    warmup_ratio=0.03,
    group_by_length = True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.001,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    gradient_accumulation_steps=1,
    lr_scheduler_type="linear",
    bf16=True,
    logging_steps=50,
    push_to_hub=True
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from trl import SFTTrainer

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    peft_config = lora_config,
    max_seq_length=128,
    eval_dataset=val_dataset,
    packing = False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2246 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
1200,2.119400,1.999602
2400,1.919400,1.911506
3600,1.960000,1.854072
4800,1.759700,1.819842
6000,1.898700,1.798655


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=6000, training_loss=1.987241766611735, metrics={'train_runtime': 5160.2097, 'train_samples_per_second': 1.163, 'train_steps_per_second': 1.163, 'total_flos': 3.260876332191744e+16, 'train_loss': 1.987241766611735, 'epoch': 2.671415850400712})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/supkon/finetuned-llama-latest/commit/bcbf847d06436963420fbefb8f134dc3dd012975', commit_message='End of training', commit_description='', oid='bcbf847d06436963420fbefb8f134dc3dd012975', pr_url=None, repo_url=RepoUrl('https://huggingface.co/supkon/finetuned-llama-latest', endpoint='https://huggingface.co', repo_type='model', repo_id='supkon/finetuned-llama-latest'), pr_revision=None, pr_num=None)

In [ ]:
model.save_pretrained("./finetuned-llama-updated")
tokenizer.save_pretrained("./finetuned-llama-updated")

('./finetuned-llama-updated/tokenizer_config.json',
 './finetuned-llama-updated/special_tokens_map.json',
 './finetuned-llama-updated/tokenizer.json')